# Assignment 2

## Instructions - Please read these instructions THOROUGHLY First! 

This is an individual homework assignment. The implication of this is that:

- You may discuss the problems in this assignment with other students in this course and your instructor/TA, but YOUR WORK MUST BE YOUR OWN.
- Do not show other students code or your own work on this assignment.
- You may consult external references, but not actively receive help from individuals not involved in this course.
- Cite all references outside of the course you used, including conversations with other students which were helpful. (This helps us give credit where it is due!). All references must use a commonly accepted reference format, for example, APA or IEEE (or another citation style of your choice).

If any of these rules seem ambiguous, please check with with your instructor for help interpreting them.

We suggest completing this assignment using the provided notebook. Each question should be answered using a SQL query (or combination or SQL queries) unless the text indicates that you may do something else. You may submit your queries embedded in Python, using SQLAlchemy or the MySQL Connector, or as plain text in Markdown.

## When you submit your work

Your submission will be graded manually. To ensure that everything goes smoothly, please follow these instructions to prepare your notebook for submission to the D2L Dropbox for Assignment 2:

- Please remove any print statments used to test your work (this is done by commenting them out)
- Please provide your solutions where asked; And please do not alter any other parts of this notebook.
- If you need to add cells to test your code please move them to the end of the notebook before submission- or you may include your commented out answers and tests in the cells provided

## Introduction

 In this assignment, we will focus familiarizing you with using SQL for data exploration, and continuing to cultivate a sense of curiosity about the datasets you encounter. We will be using a CSV File generated by the <b>City of Edmonton</b> containing Licensed Pets in the city. It is assumed that this table has been pre-cleaned (although potentially not entirely) so that you can work on the actual assignment more quickly. This assignment has four (4) parts: PARTS A, B, C, and D.
 
 To begin, start by importing the provided CSV into your own SQL database using SQLAlchemy, by filling in the lines below:

In [1]:
import pandas as pd
import sqlalchemy as sq

# read in your CSV as a dataframe
pets = pd.read_csv("pets.csv")
pets = pets.rename(columns={
    'FORWARD SORTATION AREA': 'FORWARD_SORTATION_AREA',
    'ANIMAL TYPE': 'ANIMAL_TYPE',
    'BREED': 'BREED',
    'COUNT': 'COUNT',
})

# connect to your database; include a cell at the bottom of this notebook to dispose of your engine object
# write your dataframe into a table
user = 'camila_delgado'
password = 'y8fzy2Ehmn3Xh'
host = 'datasciencedb.ucalgary.ca'
database = 'camila_delgado'

engine = sq.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(user, password, host, database))
# write your dataframe into a table

pets.to_sql(name='pets', con=engine, if_exists='replace')

# demonstrate that your import has been successful by reading your database table as a dataframe,
# and print some information (not the entire table) about your second dataframe
df_pets = pd.read_sql_table("pets", engine)
print(df_pets.head())

   index FORWARD_SORTATION_AREA ANIMAL_TYPE                BREED  COUNT
0      0                    T3L         Cat  DOMESTIC SHORT HAIR      1
1      1                    T5A         Cat  DOMESTIC SHORT HAIR    987
2      2                    T5A         Dog  RHODESIAN RIDGEBACK      3
3      3                    T5A         Dog       SIBERIAN HUSKY     50
4      4                    T5A         Dog              MALTESE     57


,index,FORWARD_SORTATION_AREA,ANIMAL_TYPE,BREED,COUNT
0,0,T3L,Cat,DOMESTIC SHORT HAIR,1
1,1,T5A,Cat,DOMESTIC SHORT HAIR,987
2,2,T5A,Dog,RHODESIAN RIDGEBACK,3
3,3,T5A,Dog,SIBERIAN HUSKY,50
4,4,T5A,Dog,MALTESE,57
...,...,...,...,...,...
5621,5621,T6X,Cat,SNOWSHOE,1
5622,5622,None,Cat,DOMESTIC MEDIUM HAIR,1
5623,5623,None,Dog,LABRADOR RETRIEVER,2
5624,5624,None,Dog,POMERANIAN,1


## Part A: Warm-up Questions (12 marks)

Answer the questions below, including the queries you used where necessary. Not all questions will require writing a SQL query to answer.

**(A1): (1 mark)**

How many records are there in total? (<b>Please show the SQL query</b>)

In [2]:
query0 = '''
SELECT COUNT(*) FROM pets;
'''

query_table0 = pd.read_sql_query(query0, engine)
print(query_table0)


   COUNT(*)
0      5626


**(A2): (1.5 mark)**

How many known areas were covered in the dataset? (<b>Please show the SQL query</b>)

In [3]:
query1 = '''
SELECT COUNT(DISTINCT `FORWARD_SORTATION_AREA`) AS Known_Areas_Count FROM pets 

'''
query_table1 = pd.read_sql_query(query1, engine)
print(query_table1)

   Known_Areas_Count
0                 39


**(A3): (1.5 mark)**

How many unknown areas were covered in the dataset? (<b>Please show the SQL query</b>)

In [4]:
query2 = """
SELECT COUNT(*) AS unknown_areas_count FROM pets
WHERE `FORWARD_SORTATION_AREA` IS NULL;
"""

query_table2 = pd.read_sql(query2, engine)
print(query_table2)

   unknown_areas_count
0                    4


**(A4): (4 marks)**

Explain what each of the following columns is used for. You may use the original page to guide your explanation (but you should cite it)
* FORWARD SORTATION AREA (1 mark)
* ANIMAL (1 mark)
* BREED (1 mark)
* COUNT (1 mark)


FORWARD SORTATION AREA: A forward sortation area (FSA) is a geographical region in which all postal codes start with the same three characters
ANIMAL TYPE: Cat or dog
BREED: A breed is a group of domestic animals with a homogeneous appearance, behavior, and other characteristics that distinguish it from other animals of the same species, and arrived at through selective breeding
COUT:

**(A5): (4 marks)**

Thinking outside the box, would it be possible to find the actual names of the community/munipalities/streets in this dataset? Why or why not? (<b>Please explain clearly and concisely.

The dataset contains the Forward Sortation Area (FSA), which represents the first three characters of Canadian postal codes. While FSAs cover specific geographic regions, they don't directly reveal the names of communities, municipalities, or streets. To determine these names, you'd typically require a mapping or lookup table correlating FSAs with their corresponding geographic names, maintained by postal authorities or GIS organizations. Without access to such data, it's not possible to derive the actual names of communities, municipalities, or streets solely from the FSA information in the dataset.

## Part B: Simple questions (10 marks) 

For these queries, run a query which provides the answer.

**(B1): (2 marks)**

List in <b>descending order</b> Five (5) Areas with the highest numbers of licensed Dogs

In [15]:
query3 = """
SELECT `FORWARD_SORTATION_AREA`, SUM(COUNT) AS Total_DOGS
FROM pets
WHERE `ANIMAL_TYPE` = 'Dog'
GROUP BY `FORWARD_SORTATION_AREA`
ORDER BY total_dogs DESC
LIMIT 5;
"""
query_table3 = pd.read_sql(query3, engine)
query_table3

,FORWARD_SORTATION_AREA,Total_DOGS
0,T5T,4537.0
1,T6L,4280.0
2,T5Y,3197.0
3,T5X,3030.0
4,T6R,2987.0


**(B2): (2 marks)**

How many Dog Breeds were captured in this dataset?

In [5]:
query4 = """
SELECT COUNT(DISTINCT BREED) AS total_dog_breeds
FROM pets
WHERE `ANIMAL_TYPE` = 'Dog';
"""

query_table4 = pd.read_sql(query4, engine)
query_table4

,total_dog_breeds
0,247


**(B3): (2 marks)**

Which Cat Breed is licensed the most? (1 Mark) How many are there in total? (1 Mark)

In [6]:
query5 = """
SELECT BREED AS most_licensed_cat_breed, SUM(COUNT) AS total_count
FROM pets
WHERE `ANIMAL_TYPE` = 'Cat'
GROUP BY BREED
ORDER BY total_count DESC
LIMIT 1;
"""

query_table5 = pd.read_sql(query5, engine)
query_table5

,most_licensed_cat_breed,total_count
0,DOMESTIC SHORT HAIR,20891.0


**(B4): (2 marks)**

Produce a table comprising of Areas and corresponding Counts for Cats of Breed types 'Himalayan' and 'Domestic Short Hair'

In [7]:
query6 = """
SELECT `FORWARD_SORTATION_AREA`, BREED, COUNT
FROM pets
WHERE `ANIMAL_TYPE` = 'Cat'
AND BREED IN ('HIMALAYAN', 'DOMESTIC SHORT HAIR')
GROUP BY `FORWARD_SORTATION_AREA`, BREED;
"""

query_table6 = pd.read_sql(query6, engine)
query_table6

,FORWARD_SORTATION_AREA,BREED,COUNT
0,None,DOMESTIC SHORT HAIR,1
1,T3L,DOMESTIC SHORT HAIR,1
2,T5A,DOMESTIC SHORT HAIR,987
3,T5A,HIMALAYAN,15
4,T5B,DOMESTIC SHORT HAIR,600
...,...,...,...
69,T6V,HIMALAYAN,8
70,T6W,DOMESTIC SHORT HAIR,366
71,T6W,HIMALAYAN,5
72,T6X,DOMESTIC SHORT HAIR,262


**(B5): (2 marks)**

What is the ratio of Cats to Dogs in this dataset?

In [8]:
query7 = """
SELECT
    SUM(CASE WHEN ANIMAL_TYPE = 'Cat' THEN COUNT ELSE 0 END) AS total_cats,
    SUM(CASE WHEN ANIMAL_TYPE = 'Dog' THEN COUNT ELSE 0 END) AS total_dogs,
    CASE 
        WHEN SUM(CASE WHEN ANIMAL_TYPE = 'Dog' THEN COUNT ELSE 0 END) = 0 THEN NULL
        ELSE CAST(SUM(CASE WHEN ANIMAL_TYPE = 'Cat' THEN COUNT ELSE 0 END) AS FLOAT) / SUM(CASE WHEN ANIMAL_TYPE = 'Dog' THEN COUNT ELSE 0 END)
    END AS cat_to_dog_ratio
FROM
    pets;


"""

query_table7 = pd.read_sql(query7, engine)
query_table7

,total_cats,total_dogs,cat_to_dog_ratio
0,33746.0,62332.0,0.541391


## Part C: Detailed analysis (20 marks)

Now consider being given a task to make sense of the distribution of licensed pets based on this dataset.

**(C1):(4 marks)**

Create two guiding questions to use in your analysis, and include them below as Markdown. As a starting point (and remember you are not limited to only these!), you may want to consider the following ideas:
- Focus on a specific set of data from the dataset that interest you
- Consider the areas with multiple breeds within the broad pet categories
- What would you say in terms of patterns of licensed pets within the specific areas of focus

**(C2): (12 marks)** 

Write at least four queries (that is, two queries for each question) which you believe will address one of your guiding questions. Clearly indicate which queries address your questions. You may wish to include a comment to explain why this query will help address your question.

**Guiding Question One: What is the most common breed by sortation area? And what animal type does this fall under?**  
This question will allow me to see the most common breed by animal type and gain a better understanding of the distribution of licenced pets 

In [9]:
query9 = """
SELECT FORWARD_SORTATION_AREA, ANIMAL_TYPE, BREED, COUNT
FROM (
    SELECT FORWARD_SORTATION_AREA, ANIMAL_TYPE, BREED, COUNT,
           ROW_NUMBER() OVER (PARTITION BY FORWARD_SORTATION_AREA ORDER BY COUNT DESC) AS rn
    FROM pets
) AS ranked
WHERE rn = 1 -- Select only the top ranked breed in each FSA
ORDER BY FORWARD_SORTATION_AREA;
"""

query_table9 = pd.read_sql(query9, engine)
query_table9

,FORWARD_SORTATION_AREA,ANIMAL_TYPE,BREED,COUNT
0,None,Dog,LABRADOR RETRIEVER,2
1,T3L,Cat,DOMESTIC SHORT HAIR,1
2,T5A,Cat,DOMESTIC SHORT HAIR,987
3,T5B,Cat,DOMESTIC SHORT HAIR,600
4,T5C,Cat,DOMESTIC SHORT HAIR,470
5,T5E,Cat,DOMESTIC SHORT HAIR,863
6,T5G,Cat,DOMESTIC SHORT HAIR,428
7,T5H,Cat,DOMESTIC SHORT HAIR,523
8,T5J,Cat,DOMESTIC SHORT HAIR,60
9,T5K,Cat,DOMESTIC SHORT HAIR,467


We learn that cats are the most common animal type and that the domestic short haired is the most popular breed 

Find out the percentage made up by each animal type (cats and dogs) in each forward sortation area (FSA) This query directly shows me a in percentages the distribution of each animal type

In [10]:
query10 = """
SELECT 
    FORWARD_SORTATION_AREA,
    ANIMAL_TYPE,
    ROUND(SUM(COUNT) * 100.0 / SUM(SUM(COUNT)) OVER (PARTITION BY FORWARD_SORTATION_AREA), 2) AS percentage
FROM pets
GROUP BY FORWARD_SORTATION_AREA, ANIMAL_TYPE
ORDER BY FORWARD_SORTATION_AREA, ANIMAL_TYPE;


"""
query_table10 = pd.read_sql(query10, engine)
query_table10.head(20)

,FORWARD_SORTATION_AREA,ANIMAL_TYPE,percentage
0,None,Cat,40.00
1,None,Dog,60.00
2,T3L,Cat,100.00
3,T5A,Cat,36.14
4,T5A,Dog,63.86
5,T5B,Cat,39.62
6,T5B,Dog,60.38
7,T5C,Cat,35.08
8,T5C,Dog,64.92
9,T5E,Cat,37.31


This query I calculated the total count of pets for each animal type and then compute the percentage of each animal type within each FSA. 

**Guiding Question Two: What is the Least common breed in the specific forward sortation area T5H?**
   

In [11]:
query11 = """
SELECT FORWARD_SORTATION_AREA, ANIMAL_TYPE, BREED, COUNT
FROM (
    SELECT FORWARD_SORTATION_AREA, ANIMAL_TYPE, BREED, COUNT,
           ROW_NUMBER() OVER (PARTITION BY FORWARD_SORTATION_AREA ORDER BY COUNT) AS rn
    FROM pets
    WHERE FORWARD_SORTATION_AREA = 'T5H' -- Filter for the specified forward sortation area
) AS ranked
WHERE rn = 1 OR COUNT = 1 -- Selects the least common breed and breeds where there is only one occurrence
ORDER BY COUNT, ANIMAL_TYPE;

"""

query_table11 = pd.read_sql(query11, engine)
query_table11

,FORWARD_SORTATION_AREA,ANIMAL_TYPE,BREED,COUNT
0,T5H,Cat,RUSSIAN BLUE,1
1,T5H,Cat,NORWEGIAN FOREST CAT,1
2,T5H,Cat,CAT OF CEYLON,1
3,T5H,Cat,BURMESE,1
4,T5H,Cat,"CROSSBREED, SMALL",1
5,T5H,Cat,"CURL, AMERICAN SHORT HAIR",1
6,T5H,Cat,BOMBAY,1
7,T5H,Cat,ANGORA,1
8,T5H,Cat,BALINESE,1
9,T5H,Cat,"CROSSBREED, MEDIUM",1


**Further investigation:**

I'm particularly focused on the forward distribution area T5H. What are the percentages of dogs, cats, and pigeons? I'm only considering data for breeds with a count of one, indicating they are the least common breeds. In T5H, there are 10 different cat breeds with a count of one, 33 dog breeds with a count of one, and one pigeon. Despite this, dogs remain more popular than both cats and pigeons in this area.

In [12]:
query12 = """
SELECT 
    FORWARD_SORTATION_AREA,
    ANIMAL_TYPE,
    COUNT(*) AS breed_count,
    ROUND((COUNT(*) * 100.0) / (SELECT COUNT(*) FROM pets WHERE FORWARD_SORTATION_AREA = 'T5H' AND COUNT = 1), 2) AS percentage
FROM pets
WHERE FORWARD_SORTATION_AREA = 'T5H' AND COUNT = 1
GROUP BY FORWARD_SORTATION_AREA, ANIMAL_TYPE;


"""
query_table12 = pd.read_sql(query12, engine)
query_table12

,FORWARD_SORTATION_AREA,ANIMAL_TYPE,breed_count,percentage
0,T5H,Cat,10,22.73
1,T5H,Dog,33,75.00
2,T5H,Pigeons,1,2.27


The above SQL query to find the least common breed in the forward sortation area T5H, including breeds where there is only one occurrence:

**(C3): (4 marks)**

What kind of data would be interesting to have to be able to make more sense of the dataset? (1 Mark) Discuss how you could use this additional information to extend one of your guiding questions. (3 Marks)

In order to gain deeper insights into the dataset, additional information such as pet size, owner demographics, and geographic features would be valuable. By incorporating owner demographics, particularly factors like income level and household size, we can extend our analysis of the distribution of licensed pets across different geographic regions. For example, correlating demographic data with pet ownership patterns can help identify trends such as higher prevalence of certain breeds in areas with higher household incomes. Specific to my guiding questions I would have liked to see if larger or smaller sized dogs were more popular and how owner demographics played a role.

## Part D: Reflection (5 marks)

In 100 to 250 words, identify a concept you have found difficult or confusing from this assignment. Reflect on how your previous learning or experience helped you to understand this concept. Provide your reflection using markdown in the cell below

Mastering new SQL commands, particularly aggregates like SUM and AVERAGE, posed a challenge in this assignment. While foundational knowledge of basic SQL provided a starting point, grasping advanced functions demanded a deeper understanding of data manipulation techniques. Consulting documentation, tutorials, and online forums proved crucial in navigating the complexities of these commands. Deconstructing intricate queries into smaller components and persistent practice were pivotal strategies for overcoming initial hurdles and enhancing SQL proficiency. 


## References

City of Edmonton (2019). Retrieved From: https://data.edmonton.ca/Demographics/Licensed-Pets-by-Breed-and-Forward-Sortation-Area-/bqmh-j34s